In [13]:
import mat73
import pandas as pd
import numpy as np
import scipy.io
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

### Data loading and preprocessing

In [14]:
import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()) and "anuja" in os.environ.get('USER'):
    DATA_DIR = 'data/'
    

In [15]:
foof = pd.read_csv(DATA_DIR+"foof2features.csv")
foof = foof.rename(columns={"C1": "IDs" ,"C2": "Intercept", "C3": "Slope"})
foof

,IDs,Intercept,Slope
0,NDARAA075AMK,0.986272,1.825774
1,NDARAA112DMH,1.486650,1.888544
2,NDARAA117NEJ,1.593155,2.095749
3,NDARAA947ZG5,0.703331,1.724831
4,NDARAA948VFH,0.918020,1.749441
...,...,...,...
2037,NDARZN277NR6,1.351549,1.996940
2038,NDARZN578YDP,1.380795,2.036327
2039,NDARZN610GTY,0.339229,1.050644
2040,NDARZN677EYE,0.781225,1.470061


In [16]:
data = scipy.io.loadmat(DATA_DIR+'x.mat')  
df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))

# sparsing
df = np.array(df).reshape(data['x'].shape)
df_sparsed = np.concatenate([np.expand_dims(df[:,:,i:i+2].mean(axis = 2), axis = 2) for i in range(0, data['x'].shape[2]-2, 2)], axis = 2)
df = pd.DataFrame(df_sparsed.reshape((df_sparsed.shape[0], -1)))

#scaling
norm = MinMaxScaler().fit(df)
df = norm.transform(df)
df = pd.DataFrame(df.reshape((df.shape[0], -1)))

columns = np.asarray([['Electrode %d - %d/2 Hz'%(i+1, j+1)] for i in range(df_sparsed.shape[1]) for j in range(df_sparsed.shape[2])])
df.columns = columns
df['IDs'] = foof['IDs']
df

,"(Electrode 1 - 1/2 Hz,)","(Electrode 1 - 2/2 Hz,)","(Electrode 1 - 3/2 Hz,)","(Electrode 1 - 4/2 Hz,)","(Electrode 1 - 5/2 Hz,)","(Electrode 1 - 6/2 Hz,)","(Electrode 1 - 7/2 Hz,)","(Electrode 1 - 8/2 Hz,)","(Electrode 1 - 9/2 Hz,)","(Electrode 1 - 10/2 Hz,)",...,"(Electrode 105 - 31/2 Hz,)","(Electrode 105 - 32/2 Hz,)","(Electrode 105 - 33/2 Hz,)","(Electrode 105 - 34/2 Hz,)","(Electrode 105 - 35/2 Hz,)","(Electrode 105 - 36/2 Hz,)","(Electrode 105 - 37/2 Hz,)","(Electrode 105 - 38/2 Hz,)","(Electrode 105 - 39/2 Hz,)",IDs
0,1.212006e-10,2.857691e-08,0.000006,0.001066,0.033290,0.143816,0.221608,0.467715,0.593581,0.401686,...,0.000007,0.000002,4.333844e-07,9.373567e-08,1.616373e-08,3.256580e-09,4.986456e-10,7.457518e-11,1.130004e-11,NDARAA075AMK
1,1.539866e-01,3.856645e-01,0.074284,0.216988,0.177267,0.161251,0.117761,0.132384,0.049898,0.004720,...,0.031636,0.009416,2.352137e-03,4.122513e-04,4.825197e-05,5.944433e-06,5.749150e-07,6.398086e-08,8.415637e-09,NDARAA112DMH
2,3.439263e-05,4.110931e-03,0.094118,0.367821,0.284890,0.164614,0.216934,0.333176,0.526883,0.511715,...,0.000311,0.000079,2.045964e-05,4.443460e-06,7.491343e-07,1.436770e-07,2.039047e-08,2.751924e-09,3.663748e-10,NDARAA117NEJ
3,2.832192e-01,3.613309e-01,0.378969,0.299813,0.247412,0.058588,0.015256,0.004514,0.003988,0.061608,...,0.158691,0.150551,1.580043e-01,1.518115e-01,1.233778e-01,1.242931e-01,1.009636e-01,8.498748e-02,7.690149e-02,NDARAA947ZG5
4,1.247342e-04,1.132987e-02,0.167700,0.368840,0.155464,0.104191,0.194459,0.288490,0.233210,0.125520,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,2.303407e-04,2.867255e-02,0.194945,0.081646,0.018422,0.076959,0.179779,0.300051,0.417245,0.459853,...,0.093399,0.058729,4.012462e-02,2.464919e-02,1.257993e-02,7.816617e-03,3.846395e-03,1.926406e-03,1.018633e-03,NDARZN277NR6
2038,1.385606e-02,2.124126e-02,0.032991,0.043456,0.048827,0.049759,0.074988,0.222227,0.444972,0.464653,...,0.016832,0.009602,5.877454e-03,3.190797e-03,1.418509e-03,7.564419e-04,3.146594e-04,1.311957e-04,5.687030e-05,NDARZN578YDP
2039,1.089647e-02,3.749309e-02,0.139003,0.289797,0.304728,0.196113,0.191950,0.342653,0.413368,0.344024,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARZN610GTY
2040,6.411911e-03,2.687961e-02,0.095551,0.241121,0.434011,0.574767,0.694741,0.815939,0.678021,0.490314,...,0.059230,0.026090,1.169173e-02,4.410850e-03,1.294309e-03,4.329110e-04,1.073561e-04,2.536768e-05,5.924775e-06,NDARZN677EYE


In [17]:
data = mat73.loadmat(DATA_DIR+'x_source.mat')  
df2 = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))

# sparsing
df2 = np.array(df2).reshape(data['x'].shape) 
df2_sparsed = np.concatenate([np.expand_dims(df2[:,:,i:i+10].mean(axis = 2), axis = 2) for i in range(0, data['x'].shape[2]-10, 10)], axis = 2)
df2 = pd.DataFrame(df2_sparsed.reshape((df2_sparsed.shape[0], -1)))

#scaling
norm = MinMaxScaler().fit(df2)
df2 = norm.transform(df2)
df2 = pd.DataFrame(df2.reshape((df2.shape[0], -1)))

df2['IDs'] = foof['IDs']
df2

,0,1,2,3,4,5,6,7,8,9,...,2643,2644,2645,2646,2647,2648,2649,2650,2651,IDs
0,3.497571e-01,4.585819e-01,2.938905e-01,3.770603e-01,0.327764,0.552059,0.462017,0.695207,0.642190,0.584085,...,0.337358,0.439658,0.437051,0.364673,0.405482,0.326733,0.428504,0.250803,0.179322,NDARAA075AMK
1,7.464767e-01,9.048637e-01,5.054438e-01,7.021925e-01,0.498947,0.342338,0.283478,0.221679,0.178758,0.178338,...,0.430404,0.296341,0.321952,0.131375,0.579138,0.478879,0.304577,0.360859,0.448476,NDARAA112DMH
2,2.204578e-01,3.562272e-01,3.895027e-01,4.181956e-01,0.495129,0.436858,0.392300,0.446757,0.528172,0.409405,...,0.404292,0.339923,0.505282,0.340129,0.203517,0.174711,0.341549,0.693619,0.545169,NDARAA117NEJ
3,6.346560e-02,1.204557e-01,1.650503e-01,7.631559e-01,0.523793,0.399243,0.267432,0.301397,0.283535,0.363612,...,0.108138,0.049740,0.033945,0.304668,0.111156,0.008791,0.005297,0.266759,0.339304,NDARAA947ZG5
4,2.758219e-01,2.255777e-01,4.828928e-01,4.704279e-01,0.330692,0.310594,0.284202,0.298111,0.245370,0.250187,...,0.164094,0.161737,0.123381,0.079693,0.069366,0.043857,0.029212,0.021894,0.013197,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2036,3.394601e-01,4.781692e-01,5.423877e-01,4.137127e-01,0.379793,0.413474,0.538537,0.457002,0.286496,0.222562,...,0.268739,0.328251,0.327389,0.292491,0.367808,0.346005,0.348895,0.399141,0.367943,NDARZN148PMN
2037,1.072605e-01,2.045245e-01,2.610531e-01,3.236518e-01,0.437417,0.432447,0.412152,0.445771,0.374603,0.386189,...,0.451919,0.240030,0.125819,0.083812,0.289439,0.378008,0.264043,0.502426,0.008403,NDARZN277NR6
2038,2.801818e-01,3.838379e-01,3.575040e-01,3.295642e-01,0.339077,0.263007,0.204378,0.308062,0.413562,0.319891,...,0.386920,0.543971,0.590438,0.544979,0.679570,0.616411,0.589517,0.634712,0.549726,NDARZN578YDP
2039,3.260273e-15,3.373362e-14,2.304057e-13,1.498324e-12,0.000009,0.142998,0.689837,0.800874,0.499946,0.248208,...,0.213887,0.230102,0.189709,0.132429,0.124579,0.085125,0.061278,0.049636,0.032335,NDARZN610GTY


In [18]:
beh = pd.read_csv(DATA_DIR+"behaviorals.csv")
print('Before:'+str(beh.shape))

most_common_disorders = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'No Diagnosis Given', 'Communication Disorder',
                         'Depressive Disorders']

# most_common_disorders = ['Other Neurodevelopmental Disorders', 'ADHD-Inattentive Type', 'ADHD-Combined Type', 'Anxiety Disorders', 'No Diagnosis Given', 'Depressive Disorders']

category_columns = ['DX_' + str(i).zfill(2) + '_Cat' for i in range(1, 11)] +\
                   ['DX_' + str(i).zfill(2) + '_Sub' for i in range(1, 11)]

# find users that have no diagnosis within these top diseases
# filtering should cahnge anything as this should also happen at a later stage
mask = None
for col in category_columns:
    mask_col = beh[col].isin(most_common_disorders)
    if mask is None:
        mask = mask_col
    else:
        mask = mask | mask_col

initial_size = beh.shape[0]
beh = beh[mask]
beh = beh.reset_index(drop=True)
new_size = beh.shape[0]
print('After:'+str(beh.shape))
print('Removing', initial_size - new_size,
      'patients as their diagnoses were very uncommon.')

Before:(3076, 177)
After:(2813, 177)
Removing 263 patients as their diagnoses were very uncommon.


In [19]:
no_diagnosis_given = 'No Diagnosis Given'

if no_diagnosis_given in most_common_disorders:
    no_diag_index = most_common_disorders.index(no_diagnosis_given)
    most_common_disorders = most_common_disorders[:no_diag_index] + \
        most_common_disorders[no_diag_index + 1:]

diagnoses_to_ids = {disorder: i for i, disorder in enumerate(most_common_disorders)}
diagnoses_to_ids

{'Attention-Deficit/Hyperactivity Disorder': 0,
 'Anxiety Disorders': 1,
 'Specific Learning Disorder': 2,
 'Autism Spectrum Disorder': 3,
 'Disruptive': 4,
 'Communication Disorder': 5,
 'Depressive Disorders': 6}

In [20]:
def get_disorder(data, row, index):
    disorder = data.iloc[row][category_columns[index]]

    if disorder == 'Neurodevelopmental Disorders':
        disorder = data.iloc[row][category_columns[index + 10]]

    return disorder

order_of_disorders = []
for k in range(beh.shape[0]):
    i = 0
    disorder = get_disorder(beh, k, i)
    disorders_patient = []
    while disorder != no_diagnosis_given and not pd.isnull(disorder):
        if disorder in diagnoses_to_ids:
            if diagnoses_to_ids[disorder] not in disorders_patient:
                disorders_patient.append(diagnoses_to_ids[disorder])
        i += 1
        if i == len(category_columns):
            break
        disorder = get_disorder(beh, k, i)

    order_of_disorders.append(disorders_patient)


In [21]:
max_len_order = np.max([len(x) for x in order_of_disorders])

# pad with a new token denoting the pad token
pad_token = len(most_common_disorders)
bod_token = len(most_common_disorders) + 1
eod_token = len(most_common_disorders) + 2

order_of_disorders = [[bod_token] + x + [eod_token] + [pad_token] * (max_len_order - len(x)) for x in order_of_disorders]

order_of_disorders = np.array(order_of_disorders)

classes = np.zeros((len(most_common_disorders),
                    beh.shape[0]), dtype=np.int32)

df_disorders = beh[category_columns]

for i, disorder in enumerate(most_common_disorders):
    mask = df_disorders.select_dtypes(include=[object]). \
        applymap(lambda x: disorder in x if pd.notnull(x) else False)

    disorder_df = df_disorders[mask.any(axis=1)]

    np.add.at(classes[i], disorder_df.index.values, 1)

behaviour_data_columns = beh.columns.values.astype(np.str)

columns_to_drop = behaviour_data_columns[
    np.flatnonzero(np.core.defchararray.find(behaviour_data_columns, 'DX') != -1)]

behaviour_data = beh.drop(columns=columns_to_drop)

for disorder, classification in zip(most_common_disorders, classes):
    behaviour_data[disorder] = classification

behaviour_data['order_diagnoses'] = list(order_of_disorders)


<ipython-input-21-48b714085ca7>:25: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  behaviour_data_columns = beh.columns.values.astype(np.str)


In [22]:
labels=behaviour_data[["IDs"]+list(most_common_disorders)]
labels

,IDs,Attention-Deficit/Hyperactivity Disorder,Anxiety Disorders,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders
0,NDARAA075AMK,0,0,0,0,0,0,0
1,NDARAA112DMH,1,0,0,0,1,0,0
2,NDARAA117NEJ,1,0,0,0,1,0,0
3,NDARAA306NT2,1,1,1,0,0,1,0
4,NDARAA504CRN,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
2808,NDARZZ007YMP,0,0,0,1,0,0,0
2809,NDARZZ740MLM,1,0,0,0,0,0,0
2810,NDARZZ810LVF,0,0,0,1,0,1,0
2811,NDARZZ830JM7,0,0,0,1,0,0,0


In [23]:
df = pd.merge(df, df2, on='IDs', how='inner')
df = pd.merge(df, foof, on='IDs', how='inner')
df = pd.merge(df, labels, on='IDs', how='inner')
df

,"(Electrode 1 - 1/2 Hz,)","(Electrode 1 - 2/2 Hz,)","(Electrode 1 - 3/2 Hz,)","(Electrode 1 - 4/2 Hz,)","(Electrode 1 - 5/2 Hz,)","(Electrode 1 - 6/2 Hz,)","(Electrode 1 - 7/2 Hz,)","(Electrode 1 - 8/2 Hz,)","(Electrode 1 - 9/2 Hz,)","(Electrode 1 - 10/2 Hz,)",...,2651,Intercept,Slope,Attention-Deficit/Hyperactivity Disorder,Anxiety Disorders,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders
0,1.212006e-10,2.857691e-08,5.960983e-06,0.001066,0.033290,0.143816,0.221608,0.467715,0.593581,0.401686,...,0.179322,0.986272,1.825774,0,0,0,0,0,0,0
1,1.539866e-01,3.856645e-01,7.428434e-02,0.216988,0.177267,0.161251,0.117761,0.132384,0.049898,0.004720,...,0.448476,1.486650,1.888544,1,0,0,0,1,0,0
2,3.439263e-05,4.110931e-03,9.411772e-02,0.367821,0.284890,0.164614,0.216934,0.333176,0.526883,0.511715,...,0.545169,1.593155,2.095749,1,0,0,0,1,0,0
3,2.832192e-01,3.613309e-01,3.789685e-01,0.299813,0.247412,0.058588,0.015256,0.004514,0.003988,0.061608,...,0.339304,0.703331,1.724831,1,0,1,1,0,0,0
4,1.247342e-04,1.132987e-02,1.676998e-01,0.368840,0.155464,0.104191,0.194459,0.288490,0.233210,0.125520,...,0.013197,0.918020,1.749441,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,8.206176e-11,1.191276e-08,9.842314e-07,0.000039,0.000736,0.006939,0.040446,0.155558,0.287815,0.314661,...,0.128389,0.788340,1.573205,1,0,0,0,0,0,0
1831,3.062358e-04,1.130883e-03,3.834623e-03,0.009994,0.020117,0.032255,0.051099,0.085125,0.108455,0.129148,...,0.333760,1.423824,1.381199,0,0,0,0,0,0,0
1832,1.556547e-05,2.900630e-04,3.548476e-03,0.023882,0.088943,0.189421,0.293481,0.445277,0.644915,0.577096,...,0.367943,0.168009,0.205704,0,1,0,0,0,0,0
1833,2.303407e-04,2.867255e-02,1.949453e-01,0.081646,0.018422,0.076959,0.179779,0.300051,0.417245,0.459853,...,0.008403,1.351549,1.996940,1,1,0,0,0,0,1


### Data Split

In [24]:
x = df[df.columns.difference(['IDs']+most_common_disorders)]
y = df[most_common_disorders]

# summarize dataset shape
print(x.shape, y.shape)

(1835, 6749) (1835, 7)


In [25]:
train_features, test_features, train_labels, test_labels = train_test_split(x, y, test_size=0.3, shuffle=True)

### AutoEncoder for Feature Extraction

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical

n_inputs = train_features.shape[1]

visible = Input(shape=(n_inputs,))
e = Dense(n_inputs*2)(visible)
e = BatchNormalization()(e)
e = LeakyReLU()(e)
e = Dense(n_inputs)(e)
e = BatchNormalization()(e)
e = LeakyReLU()(e)
n_bottleneck = round(float(n_inputs) / 2.0)
bottleneck = Dense(n_bottleneck)(e)

d = Dense(n_inputs)(bottleneck)
d = BatchNormalization()(d)
d = LeakyReLU()(d)

d = Dense(n_inputs*2)(d)
d = BatchNormalization()(d)
d = LeakyReLU()(d)

output = Dense(7, activation='linear')(d)

model = Model(inputs=visible, outputs=output)
model.compile(optimizer='adam', loss='mse')
plot_model(model, 'autoencoder_compress.png', show_shapes=True)

history = model.fit(train_features, train_labels, epochs=50, batch_size=16, verbose=2)
encoder = Model(inputs=visible, outputs=bottleneck)

plot_model(encoder, 'encoder_compress.png', show_shapes=True)
encoder.save('autoencoder.h5')

2021-11-22 17:57:32.338503: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-22 17:57:32.369415: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-22 17:57:32.370113: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:06.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-11-22 17:57:32.370336: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-11-22 17:57:32.371567: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-22 17:57:32.372921: I tensorflow/stream_executor/platform/default/d

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
Epoch 1/50


2021-11-22 17:57:34.039013: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


81/81 - 6s - loss: 34.2378
Epoch 2/50
81/81 - 6s - loss: 0.2265
Epoch 3/50
81/81 - 6s - loss: 0.2321
Epoch 4/50
81/81 - 6s - loss: 0.2620
Epoch 5/50
81/81 - 6s - loss: 0.2528
Epoch 6/50
81/81 - 6s - loss: 0.2676
Epoch 7/50
81/81 - 6s - loss: 0.2821
Epoch 8/50
81/81 - 6s - loss: 0.2815
Epoch 9/50
81/81 - 6s - loss: 0.2712
Epoch 10/50
81/81 - 6s - loss: 0.3251
Epoch 11/50
81/81 - 6s - loss: 0.3141
Epoch 12/50
81/81 - 6s - loss: 0.3357
Epoch 13/50
81/81 - 6s - loss: 0.3129
Epoch 14/50
81/81 - 6s - loss: 0.3603
Epoch 15/50
81/81 - 6s - loss: 0.4269
Epoch 16/50
81/81 - 6s - loss: 0.3723
Epoch 17/50
81/81 - 6s - loss: 0.3712
Epoch 18/50
81/81 - 6s - loss: 0.3746
Epoch 19/50
81/81 - 6s - loss: 0.3587
Epoch 20/50
81/81 - 6s - loss: 0.3850
Epoch 21/50
81/81 - 6s - loss: 0.3521
Epoch 22/50
81/81 - 6s - loss: 0.3773
Epoch 23/50
81/81 - 6s - loss: 0.3591
Epoch 24/50
81/81 - 6s - loss: 0.4133
Epoch 25/50
81/81 - 6s - loss: 0.3518
Epoch 26/50
81/81 - 6s - loss: 0.3600
Epoch 27/50
81/81 - 6s - loss: 

In [15]:
# encode the data
encoder= load_model('autoencoder.h5', compile=False)

train_features = encoder.predict(train_features)
test_features = encoder.predict(test_features)

### Metrics

In [98]:
from sklearn.metrics import hamming_loss, accuracy_score
import sklearn.metrics as skm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

def brier_multi(targets, probs):
    return np.mean(np.sum((probs - targets)**2, axis=1))

def evaluate(y_test, y_pred_prob, brier=True):
    y_pred = y_pred_prob.round()
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Hamming Loss:", hamming_loss(y_test, y_pred))
    if brier:
        print("Brier Score:", brier_multi(y_test, y_pred_prob))
    print("Classification Report:\n", skm.classification_report(y_test,y_pred, zero_division=1))
    print("Confusion matrix:\n", skm.multilabel_confusion_matrix(y_test, y_pred))

## Models

### Multi Output Classifier

In [99]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC as svm
from sklearn.multioutput import MultiOutputClassifier

forest = RandomForestClassifier(random_state=1)
lg = LogisticRegression()
svm = svm()
models = [lg, forest, svm]

for model in models:

    multi_output_model = MultiOutputClassifier(model, n_jobs=-1)
    multi_output_model.fit(train_features, train_labels)
    predicted_labels = multi_output_model.predict(test_features)
    print(str(model)+':')
    evaluate(test_labels, predicted_labels, brier=False)

LogisticRegression():
Accuracy: 0.10344827586206896
Hamming Loss: 0.2756028001037075
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.64      0.65       369
           1       0.40      0.30      0.34       203
           2       0.27      0.15      0.19       140
           3       0.22      0.06      0.10        94
           4       0.10      0.03      0.05        92
           5       0.16      0.09      0.12        76
           6       0.26      0.09      0.13        58

   micro avg       0.48      0.33      0.39      1032
   macro avg       0.30      0.19      0.23      1032
weighted avg       0.41      0.33      0.35      1032
 samples avg       0.59      0.43      0.36      1032

Confusion matrix:
 [[[ 58 124]
  [134 235]]

 [[258  90]
  [142  61]]

 [[353  58]
  [119  21]]

 [[436  21]
  [ 88   6]]

 [[433  26]
  [ 89   3]]

 [[439  36]
  [ 69   7]]

 [[479  14]
  [ 53   5]]]
RandomForestClassifier(random_state=1):

### MLP

In [100]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def get_mlp(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

n_inputs, n_outputs = train_features.shape[1],train_labels.shape[1]
mlp = get_mlp(n_inputs, n_outputs)
mlp.fit(train_features, train_labels, verbose=1, epochs=100)

Epoch 1/100
41/41 [==============================] - 1s 4ms/step - loss: 0.5245
Epoch 2/100
41/41 [==============================] - 0s 3ms/step - loss: 0.4947
Epoch 3/100
41/41 [==============================] - 0s 4ms/step - loss: 0.4898
Epoch 4/100
41/41 [==============================] - 0s 5ms/step - loss: 0.4830
Epoch 5/100
41/41 [==============================] - 0s 5ms/step - loss: 0.4798
Epoch 6/100
41/41 [==============================] - 0s 5ms/step - loss: 0.4808
Epoch 7/100
41/41 [==============================] - 0s 6ms/step - loss: 0.4730
Epoch 8/100
41/41 [==============================] - 0s 6ms/step - loss: 0.4670
Epoch 9/100
41/41 [==============================] - 0s 6ms/step - loss: 0.4660
Epoch 10/100
41/41 [==============================] - 0s 5ms/step - loss: 0.4645
Epoch 11/100
41/41 [==============================] - 0s 6ms/step - loss: 0.4564
Epoch 12/100
41/41 [==============================] - 0s 5ms/step - loss: 0.4537
Epoch 13/100
41/41 [=================

In [101]:
predicted_labels_mlp = mlp.predict(test_features)
evaluate(test_labels, predicted_labels_mlp)

Accuracy: 0.0780399274047187
Hamming Loss: 0.29038112522686027
Brier Score: 1.5445178994569044
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.73      0.70       369
           1       0.39      0.38      0.39       203
           2       0.21      0.03      0.05       140
           3       0.17      0.28      0.21        94
           4       0.19      0.09      0.12        92
           5       0.10      0.05      0.07        76
           6       0.22      0.09      0.12        58

   micro avg       0.45      0.38      0.41      1032
   macro avg       0.28      0.24      0.24      1032
weighted avg       0.40      0.38      0.38      1032
 samples avg       0.56      0.48      0.38      1032

Confusion matrix:
 [[[ 50 132]
  [ 99 270]]

 [[225 123]
  [125  78]]

 [[396  15]
  [136   4]]

 [[332 125]
  [ 68  26]]

 [[424  35]
  [ 84   8]]

 [[440  35]
  [ 72   4]]

 [[475  18]
  [ 53   5]]]


### Binary Relevance
ignores the possible correlations between class labels

In [102]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

classifier = BinaryRelevance(GaussianNB())
classifier.fit(test_features, test_labels)

BinaryRelevance(classifier=GaussianNB(), require_dense=[True, True])

In [103]:
predicted_labels_br = classifier.predict_proba(test_features)
evaluate(test_labels, predicted_labels_br.todense())

Accuracy: 0.07622504537205081
Hamming Loss: 0.3411978221415608
Brier Score: 2.380635109465426
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.80      0.78       369
           1       0.46      0.82      0.59       203
           2       0.35      0.82      0.49       140
           3       0.41      0.64      0.50        94
           4       0.25      0.92      0.40        92
           5       0.28      0.74      0.41        76
           6       0.26      0.74      0.39        58

   micro avg       0.43      0.80      0.56      1032
   macro avg       0.40      0.78      0.51      1032
weighted avg       0.51      0.80      0.59      1032
 samples avg       0.44      0.82      0.50      1032

Confusion matrix:
 [[[ 89  93]
  [ 74 295]]

 [[153 195]
  [ 36 167]]

 [[193 218]
  [ 25 115]]

 [[372  85]
  [ 34  60]]

 [[209 250]
  [  7  85]]

 [[332 143]
  [ 20  56]]

 [[372 121]
  [ 15  43]]]


### Classfier Chains

In [104]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression

classifier = ClassifierChain(LogisticRegression())
classifier.fit(train_features, train_labels)
# we should optimise this a little

/home/anujanegi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/anujanegi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

ClassifierChain(classifier=LogisticRegression(), require_dense=[True, True])

In [105]:
predicted_labels_cc = classifier.predict_proba(test_features)
evaluate(test_labels, predicted_labels_cc.todense())

Accuracy: 0.10163339382940109
Hamming Loss: 0.27586206896551724
Brier Score: 1.5202853169271076
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.63      0.64       369
           1       0.40      0.30      0.34       203
           2       0.27      0.15      0.19       140
           3       0.23      0.07      0.11        94
           4       0.07      0.02      0.03        92
           5       0.18      0.08      0.11        76
           6       0.30      0.12      0.17        58

   micro avg       0.48      0.33      0.39      1032
   macro avg       0.30      0.20      0.23      1032
weighted avg       0.41      0.33      0.35      1032
 samples avg       0.59      0.43      0.36      1032

Confusion matrix:
 [[[ 58 124]
  [137 232]]

 [[256  92]
  [142  61]]

 [[354  57]
  [119  21]]

 [[434  23]
  [ 87   7]]

 [[431  28]
  [ 90   2]]

 [[447  28]
  [ 70   6]]

 [[477  16]
  [ 51   7]]]


### Label Powerset
takes correlations into account!

In [106]:
from skmultilearn.problem_transform import LabelPowerset

classifier = LabelPowerset(LogisticRegression())
classifier.fit(train_features, train_labels)

LabelPowerset(classifier=LogisticRegression(), require_dense=[True, True])

In [107]:
predicted_labels_lp = classifier.predict_proba(test_features)
evaluate(test_labels, predicted_labels_lp.todense())

Accuracy: 0.16333938294010888
Hamming Loss: 0.21908218822919368
Brier Score: 1.1430201072638624
Classification Report:
               precision    recall  f1-score   support

           0       0.67      1.00      0.80       369
           1       1.00      0.00      0.00       203
           2       1.00      0.00      0.00       140
           3       1.00      0.00      0.00        94
           4       1.00      0.00      0.00        92
           5       1.00      0.00      0.00        76
           6       1.00      0.00      0.00        58

   micro avg       0.67      0.36      0.47      1032
   macro avg       0.95      0.14      0.11      1032
weighted avg       0.88      0.36      0.29      1032
 samples avg       0.67      0.47      0.45      1032

Confusion matrix:
 [[[  0 182]
  [  0 369]]

 [[348   0]
  [203   0]]

 [[411   0]
  [140   0]]

 [[457   0]
  [ 94   0]]

 [[459   0]
  [ 92   0]]

 [[475   0]
  [ 76   0]]

 [[493   0]
  [ 58   0]]]


### Multi Label KNN

In [108]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix

mlknn = MLkNN(k=10)

x_train = lil_matrix(train_features).toarray()
y_train = lil_matrix(train_labels).toarray()
x_test = lil_matrix(test_features).toarray()

mlknn.fit(x_train, y_train)

/home/anujanegi/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_neighbors=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


MLkNN()

In [109]:
predicted_labels_mlknn = mlknn.predict_proba(x_test)
evaluate(test_labels, predicted_labels_mlknn.todense())

Accuracy: 0.10707803992740472
Hamming Loss: 0.26756546538760695
Brier Score: 1.649097527784518
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.00      0.00       369
           1       1.00      0.00      0.00       203
           2       1.00      0.00      0.00       140
           3       1.00      0.00      0.00        94
           4       1.00      0.00      0.00        92
           5       1.00      0.00      0.00        76
           6       1.00      0.00      0.00        58

   micro avg       1.00      0.00      0.00      1032
   macro avg       1.00      0.00      0.00      1032
weighted avg       1.00      0.00      0.00      1032
 samples avg       1.00      0.11      0.11      1032

Confusion matrix:
 [[[182   0]
  [369   0]]

 [[348   0]
  [203   0]]

 [[411   0]
  [140   0]]

 [[457   0]
  [ 94   0]]

 [[459   0]
  [ 92   0]]

 [[475   0]
  [ 76   0]]

 [[493   0]
  [ 58   0]]]
